# OpenAI 유틸리티 기능에 대한 심층 가이드

API 구성 관리는 특히 여러 모델과 API 버전을 다룰 때 까다로울 수 있습니다. 제공되는 유틸리티 함수는 사용자가 이러한 구성을 효과적으로 관리할 수 있도록 도와줍니다. API 키 및 기타 민감한 데이터가 안전하게 저장되어 있는지 확인하세요. 로컬 개발을 위해 키를 '.txt' 또는 '.env' 파일이나 환경 변수에 저장할 수 있습니다. API 키를 공개적으로 노출하지 마세요. 키 파일을 리포지토리에 로컬로 저장해야 하는 경우(저장하지 않는 것이 좋습니다), 키 파일 경로가 '.gitignore` 파일에 추가되어 있는지 확인하세요.

#### 단계:
1. OpenAI 및 선택적으로 Azure OpenAI(또는 다른 공급자)에서 API 키를 가져옵니다.
2. 둘 중 하나를 사용하여 안전하게 저장합니다:
    - 환경 변수: 셸에서 `export OPENAI_API_KEY='your-key'`를 사용합니다.
    - 텍스트 파일: `key_openai.txt` 파일에 키를 저장합니다.
    - 환경 파일: `.env` 파일에 키를 저장합니다(예: `OPENAI_API_KEY=sk-********************`).

---

**TL;DR:** <br>
사용 사례에 따라 `config_list`를 생성하는 방법은 여러 가지가 있습니다:

- get_config_list`: 주로 제공된 API 키에서 API 호출에 대한 구성을 생성합니다.
- CONFIG_LIST_OPENAI_AOAI`: 환경 변수 또는 로컬 파일에서 API 키를 소싱하여 Azure OpenAI 및 OpenAI 엔드포인트를 모두 사용하여 구성 목록을 작성합니다.
- CONFIG_LIST_From_JSON`: 주어진 기준에 따라 구성을 필터링할 수 있는 유연성과 함께 환경 변수 또는 로컬 JSON 파일에서 JSON 구조에서 구성을 로드합니다.
- CONFIG_LIST_From_MODELS`: 제공된 모델 목록을 기반으로 구성을 생성하며, 각 구성을 수동으로 지정하지 않고 특정 모델을 타겟팅할 때 유용합니다.
- CONFIG_LIST_From_DOTENV`: .env` 파일에서 구성 목록을 생성하여 단일 파일에서 여러 API 구성 및 키를 관리할 수 있는 통합된 방법을 제공합니다.

#### `config_list`란 무엇인가요?
아래 예시와 같이 어시스턴트를 인스턴스화할 때 `config_list`가 전달됩니다. 이는 `AssistantAgent`가 액세스할 수 있는 모델 또는 구성을 알려주는 데 사용됩니다:
``python

assistant = AssistantAgent(
    name="assistant",
    llm_config={
        "요청_타임아웃": 600,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0,
    },
)
```

OpenAI의 GPT 모델을 활용하는 지능형 어시스턴트를 생각해 봅시다. 사용자 요청에 따라 필요할 수도 있습니다:

- 창의적인 콘텐츠 생성(gpt-4 사용).
- 일반적인 쿼리에 응답(gpt-3.5-turbo 사용).

작업마다 다른 모델이 필요할 수 있으며, `config_list`는 지능형 어시스턴트의 효율적인 운영을 위해 적절한 모델 구성을 동적으로 선택하고 API 키, 엔드포인트 및 버전을 관리하는 데 도움을 줍니다. 요약하면, `config_list`는 다양한 설정과 OpenAI API와의 상호작용을 관리하여 에이전트가 효율적이고 안정적이며 최적으로 작동할 수 있도록 지원함으로써 에이전트의 적응력과 기능을 향상시킵니다.

In [ ]:
# ! pip install pyautogen

In [ ]:
import autogen 

## get_config_list

API 호출에 대한 구성을 생성하는 데 사용됩니다.

In [ ]:
api_keys = ["YOUR_OPENAI_API_KEY"]
api_bases = None  # You can specify API base URLs if needed. eg: localhost:8000
api_type = "openai"  # Type of API, e.g., "openai" or "aoai".
api_version = None  # Specify API version if needed.

config_list = autogen.get_config_list(
    api_keys,
    api_bases=api_bases,
    api_type=api_type,
    api_version=api_version
)

print(config_list)

## config_list_openai_aoai

이 메서드는 Azure OpenAI 엔드포인트 및 OpenAI 엔드포인트를 사용하여 구성 목록을 만듭니다. 환경 변수 또는 로컬 텍스트 파일에서 API 키와 베이스를 추출하려고 시도합니다.

단계
- OpenAI API 키를 저장합니다:
    - 환경 변수: `OPENAI_API_KEY`
    - 또는 로컬 파일: `key_openai.txt`
- Azure OpenAI API 키를 저장합니다:
    - 환경 변수: `azure_openai_api_key`
    - 또는 로컬 파일: `key_aoai.txt`(한 줄에 하나씩 여러 키 지원)
- Azure OpenAI API 베이스를 저장합니다:
    - 환경 변수 `azure_openai_api_base`
    - 또는 로컬 파일: `base_aoai.txt`(한 줄당 하나씩 여러 개의 베이스 지원)

In [ ]:
config_list = autogen.config_list_openai_aoai(
    key_file_path=".",
    openai_api_key_file="key_openai.txt",
    aoai_api_key_file="key_aoai.txt",
    aoai_api_base_file="base_aoai.txt",
    exclude=None # The API type to exclude, eg: "openai" or "aoai".
)

## config_list_from_json

이 메서드는 환경 변수 또는 JSON 파일에서 구성을 로드합니다. 이 메서드는 사용자가 특정 기준에 따라 구성을 필터링할 수 있도록 하여 유연성을 제공합니다.

단계:
- JSON 구성을 설정합니다:
    1. 환경 변수 `OAI_CONFIG_LIST`에 구성을 유효한 JSON 문자열로 저장합니다.
    2. 또는 `OAI_CONFIG_LIST.json`이라는 로컬 JSON 파일에 구성을 저장합니다.
    3. 로컬 리포지토리의 `.gitignore` 파일에 `OAI_CONFIG_LIST`를 추가합니다.

JSON 구조는 다음과 같아야 합니다:

```json
# OAI_CONFIG_LIST 파일 예제
[
    {
        "모델": "gpt-4",
        "api_key": "YOUR_OPENAI_API_KEY"
    },
    {
        "model": "gpt-3.5-turbo",
        "api_key": "YOUR_OPENAI_API_KEY",
        "api_version": "2023-03-01-preview"
    }
]

```


In [ ]:
config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",  # or OAI_CONFIG_LIST.json if file extension is added
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    }
)

#### `필터_딕`이란 무엇인가요?

autogen.config_list_from_json` 함수의 z 파라미터는 환경 변수 또는 JSON 파일에서 로드된 구성을 지정한 조건에 따라 선택적으로 필터링하는 데 사용됩니다. 정의한 조건과 일치하는 구성만 선택하도록 조건을 정의할 수 있습니다.

예를 들어 어시스턴트 에이전트를 LLM 유형으로만 구성하고 싶다고 가정해 보겠습니다. 아래 예시를 보면, `OAI_CONFIG_LIST`에 "gpt-3.5-turbo"와 "gpt-4"가 있지만, 이 에이전트는 다음과 같은 조건만 사용하도록 구성됩니다.

In [ ]:
cheap_config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",  
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
        }
    }
)

costly_config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST", 
    filter_dict={
        "model": {
            "gpt-4",
        }
    }
)

# Assistant using GPT 3.5 Turbo
assistant_one = autogen.AssistantAgent(
    name="3.5-assistant",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": cheap_config_list,
        "temperature": 0,
    },
)

# Assistant using GPT 4
assistant_two = autogen.AssistantAgent(
    name="4-assistant",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": costly_config_list,
        "temperature": 0,
    },
)

앞서 설정한 `OAI_CONFIG_LIST`를 사용하면 필터링할 항목이 많지 않습니다. 하지만 프로젝트의 복잡성이 커지고 다양한 목적으로 여러 모델을 관리해야 하는 경우 `filter_dict`가 얼마나 유용할 수 있는지 알 수 있습니다.

좀 더 복잡한 필터링 기준은 다음과 같을 수 있습니다: 다양한 에이전트를 생성하는 데 사용되는 여러 모델이 `OAI_CONFIG_LIST`에 있다고 가정합니다. API 버전 `"2023-03-01-preview"`를 사용하는 `gpt-4`에 대한 구성을 로드하고 `api_type`을 `aoai`로 설정하고 싶다고 가정하면 다음과 같이 `filter_dict`를 구성할 수 있습니다:

In [ ]:
config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    filter_dict = {
        "model": {
            "gpt-4"
        },
        "api_version": {
            "2023-03-01-preview"
        },
        "api_type": 
        ["aoai"]
    },
)

## config_list_from_models

이 메서드는 제공된 모델 목록을 기반으로 구성을 생성합니다. 특정 모델을 염두에 두고 있고 각 구성을 수동으로 지정하고 싶지 않을 때 유용합니다.

단계:
- 방법 1과 유사하게 API 키와 베이스를 환경 변수 또는 '.txt' 파일에 저장합니다.

In [ ]:
config_list = autogen.config_list_from_models(
    key_file_path = ".",
    openai_api_key_file = "key_openai.txt",
    aoai_api_key_file = "key_aoai.txt",
    aoai_api_base_file = "base_aoai.txt",
    exclude="aoai",
    model_list = None,
    model_list=["gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"],
)

## config_list_from_dotenv

OpenAI 전용 구성을 사용하는 대신 모든 키를 `.env` 파일과 같은 단일 위치에 보관하고 싶다면 `config_list_from_dotenv`를 사용할 수 있습니다. 이렇게 하면 복잡한 `OAI_CONFIG_LIST` 파일을 만들지 않고도 편리하게 구성 목록을 생성할 수 있습니다.

model_api_key_map` 파라미터는 모델 이름을 해당 API 키가 저장된 `.env` 파일의 환경 변수 이름에 매핑하는 딕셔너리입니다. 이를 통해 코드가 각 모델에 사용할 API 키를 알 수 있습니다.

제공하지 않으면 기본적으로 `gpt-4`의 경우 `OPENAI_API_KEY`를, `gpt-3.5-turbo`의 경우 `OPENAI_API_KEY`를 사용합니다.

```python
    # 기본 키 맵
    MODEL_API_KEY_MAP = {
        "gpt-4": "openai_api_key",
        "gpt-3.5-turbo": "openai_api_key",
    }
```

다음은 `.env` 파일 예시입니다:

```bash
OPENAI_API_KEY=sk-*********************
HUGGING_FACE_API_KEY=**************************
ANOTHER_API_KEY=1234567890234567890
```

In [1]:
import autogen

config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env', # If None the function will try to find in the working directory
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    }
)

config_list

[{'api_key': 'sk-*********************', 'model': 'gpt-4'},
 {'api_key': 'sk-*********************', 'model': 'gpt-3.5-turbo'}]

In [2]:
# gpt-3.5-turbo will default to OPENAI_API_KEY
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env', # If None the function will try to find in the working directory
    model_api_key_map={
        "gpt-4": "ANOTHER_API_KEY",  # String or dict accepted
    },
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    }
)

config_list

[{'api_key': '1234567890234567890', 'model': 'gpt-4'},
 {'api_key': 'sk-*********************', 'model': 'gpt-3.5-turbo'}]

In [3]:
# example using different environment variable names
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env',
    model_api_key_map={
        "gpt-4": "OPENAI_API_KEY",
        "vicuna": "HUGGING_FACE_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4",
            "vicuna",
        }
    }
)

config_list

[{'api_key': 'sk-*********************', 'model': 'gpt-4'},
 {'api_key': '**************************', 'model': 'vicuna'}]

또한 문자열 값을 구성에 확장된 사전으로 대체하여 API에 대한 추가 구성을 제공할 수도 있습니다. 아래 예시는 기본적으로 `openai`에 `gpt-4`를 사용하고, `aoai`에 추가 구성으로 `gpt-3.5-turbo`를 사용하는 예시를 보여 줍니다.

In [4]:
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env',
    model_api_key_map={
        "gpt-4": "OPENAI_API_KEY",
        "gpt-3.5-turbo": {
            "api_key_env_var": "ANOTHER_API_KEY",
            "api_type": "aoai",
            "api_version": "v2",
            "api_base": "https://api.someotherapi.com"
        }
    },
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    }
)

config_list

[{'api_key': 'sk-*********************', 'model': 'gpt-4'},
 {'api_key': '1234567890234567890',
  'api_base': 'https://api.someotherapi.com',
  'api_type': 'aoai',
  'api_version': 'v2',
  'model': 'gpt-3.5-turbo'}]